## Import

In [1]:
from tqdm import tnrange
from tqdm import tqdm
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

import time
import pandas as pd
import numpy as np
import re
import requests

## Function

In [2]:
def find_num(x) :
    try : output = int(x)
    except : output = "remove"
    return output

In [139]:
def cleansing_text(x) :
    tmp_text = str(x[0])
    output = re.sub("<.*?>","",tmp_text)
    output = re.sub("[\n\t▶◆▦■→&◇lt;ⓒ]","",output)
    output = re.sub("\\[.*?\\]","",output)
    output = re.sub("\xa0","",output)
    output = re.sub("@.*","",output)
    output = re.sub("☞.*","",output)
    output = re.sub("\'","",output)
    
    output = re.sub("모바일로 읽는.*","",output)
    output = re.sub("(머니마켓).*","",output)
    output = re.sub("이데일리ON.*","",output)
    output = re.sub("뉴스의 새 시대, 연합뉴스.*","",output)
    output = re.sub("MBA도 모바일로 공부한다..*","",output)
    output = re.sub("헤럴드 생생뉴스.*","",output)
    output = re.sub("데일리안.*","",output)
    output = re.sub("• .*","",output)
    output = re.sub("한경닷컴.*","",output)
    
    output = re.sub("                     \\(","",output)
    output = re.sub("    ","",output)
    output = re.sub("   ","",output)
    output = re.sub("  ","",output)
    return output

## Crawling

* 주요 뉴스 크롤링

#### news_inform

In [4]:
base_url = "https://finance.naver.com"
csv_save_path = "/home/ubunt/4-2/stock price/data/news_csv/"

In [4]:
years = [str(x) for x in range(2010,2020)]
months = [str(x) for x in range(1,13)]
days = [str(x) for x in range(1,32)]
select_days = []
for y in years :
    for m in months :
        for d in days :
            select_days.append(y + "-" + m + "-" + d)

In [5]:
start_index = 0

In [11]:
for day in tqdm_notebook(select_days[start_index:]) :
    url = "https://finance.naver.com/news/mainnews.nhn?date={}".format(day)
    res = requests.get(url)
    if res.status_code == 200 : _
    else : print("error :", day)
    soup = BeautifulSoup(res.text,"html.parser")
    
    # 페이지 수
    tmp_pages = soup.select("table.Nnavi tr > td")
    pages = [find_num(x.find("a").text) for x in tmp_pages]
    try : pages.remove("remove")
    except : _
        
    for page_index in pages :
        recent_url = url + "&page={}".format(page_index)
        # 제목 & url
        tmp_titles = soup.select('div.mainNewsList li.block1 dd.articleSubject a')
        titles_append = soup.select('div.mainNewsList li.block1 dt.articleSubject a')
        sub_titles = soup.select("dd.relArticle a")

        page_title = [x.text for x in tmp_titles]
        page_url = [base_url + x.get("href") for x in tmp_titles]
        page_title.extend([x.text for x in titles_append])
        page_url.extend([base_url + x.get("href") for x in titles_append])
        page_title.extend([x.text for x in sub_titles])
        page_url.extend([base_url + x.get("href") for x in sub_titles])

        # 신문사 & 발행 일자
        tmp_corp = soup.select('span.press')
        tmp_date = soup.select('span.wdate')

        page_corp = [x.text for x in tmp_corp]
        page_date = [x.text for x in tmp_date]
        
        page_df = pd.DataFrame({"date" : page_date, "title" : page_title, "corp" : page_corp, "url" : page_url})
    
    if start_index == 0 :
        news_inform = page_df
    else :
        news_inform = news_inform.append(page_df)
    
    start_index += 1

In [23]:
news_inform.index = [x for x in range(news_inform.shape[0])]
news_inform.to_csv(csv_save_path + "news_inform_2019092721.csv", index = False)

#### news text

In [5]:
news_inform = pd.read_csv(csv_save_path + "news_inform_2019092721.csv")

In [145]:
start_idx = 0

In [ ]:
for news_url in tqdm_notebook(news_inform.url[start_idx:]) :
    res = requests.get(news_url)
    assert res.status_code == 200
    
    soup = BeautifulSoup(res.text,"html.parser")
    news_text = cleansing_text(soup.select("div.articleCont"))
    tmp_df = pd.DataFrame({"text" : [news_text], "url" : [news_url]})
    
    if start_idx == 0 :
        news_text_df = tmp_df
    else :
        news_text_df = news_text_df.append(tmp_df)
    
    start_idx += 1

[]